In [10]:

import json
import os
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split, GridSearchCV, RandomizedSearchCV
from sklearn.metrics import accuracy_score, mean_squared_error
from sklearn.linear_model import LogisticRegression, LinearRegression, Ridge
from sklearn import tree
from sklearn.ensemble import RandomForestClassifier, RandomForestRegressor
from sklearn.preprocessing import MinMaxScaler, StandardScaler, LabelEncoder
from sklearn.impute import SimpleImputer
import joblib
import ast
import seaborn as sns

In [2]:
df = pd.read_csv('stroke.csv', index_col=0)

In [3]:
def select_model(model):
    if model=="Logistic Regression":
        ML_model = LogisticRegression()
    elif model == "Decision Tree":
        ML_model = tree.DecisionTreeClassifier()
    elif model == "Random Forest Classificator":
        ML_model = RandomForestClassifier()
    elif model == "Linear Regression":
        ML_model = LinearRegression()
    elif model == "Regression Tree":
        ML_model = tree.DecisionTreeRegressor()
    elif model == "Ridge Regression":
        ML_model = Ridge()
    elif model == "Random Forest Regressor":
        ML_model = RandomForestRegressor()
    return ML_model

In [4]:
def save_model(model, model_name, folder_name='saved_models'):
    # Create the folder if it doesn't exist
    if not os.path.exists(folder_name):
        os.makedirs(folder_name)

    # Define the file path
    file_path = os.path.join(folder_name, model_name + '.pkl')

    # Save the model using joblib
    joblib.dump(model, file_path)
    print(f"Model saved at: {file_path}")

In [5]:
# Custom imputation using random values between min and max for each column
def random_impute(X):
    for i in range(X.shape[1]):
        col = X[:, i]
        missing = np.isnan(col)
        col_min, col_max = np.nanmin(col), np.nanmax(col)
        col[missing] = np.random.uniform(col_min, col_max, size=missing.sum())
    return X
        
def clean_data(X, imputation_strategy='most_frequent', scaling_method='minmax'):
    # Separate numerical and non-numerical columns
    numerical_cols = X.select_dtypes(include=[np.number]).columns
    categorical_cols = X.select_dtypes(exclude=[np.number]).columns

    # Handle missing values based on the chosen imputation strategy
    if imputation_strategy in ['mean', 'median','most_frequent']:
        imputer = SimpleImputer(strategy=imputation_strategy)
        X_imputed = pd.DataFrame(imputer.fit_transform(X), columns=X.columns)
    elif imputation_strategy == 'random':        
        X_imputed = random_impute(X.copy())  # Apply random imputation before scaling
    else:
        raise ValueError("Invalid imputation strategy. Choose 'mean', 'median', 'most_frequent' or 'random'.")
    
    # Encode non-numerical columns with integer encoding
    X_imputed[categorical_cols] = X_imputed[categorical_cols].apply(LabelEncoder().fit_transform)


    # Apply scaling based on the chosen scaling method
    if scaling_method == 'minmax':
        scaler = MinMaxScaler()
    elif scaling_method == 'standard':
        scaler = StandardScaler()
    else:
        raise ValueError("Invalid scaling method. Choose 'minmax' or 'standard'.")
    
    # Fit and transform the data
    X_scaled = scaler.fit_transform(X_imputed)
    X_cleaned = pd.DataFrame.from_records(data=X_scaled, columns=X.columns)
    return X_cleaned

In [6]:
def train_model(model, train_test_split, train_config):
    
    X_train, X_test, y_train, y_test = train_test_split
    param_grid = model["params"]
    method = select_model(model["model_type"])
    metric = train_config["metric"]
    folds = train_config["CV"]
    if folds == False:
        folds = None 
        
    if "n_iter" in model.keys():
        # Create a RandomizedSearch object with cross-validation    
        searchCV = RandomizedSearchCV(method, param_grid, n_iter=model["n_iter"], cv=folds, n_jobs=-1, scoring=metric, random_state=train_config["seed"], return_train_score=True)
    else:
        for p in param_grid:
            if not isinstance(param_grid[p], list):
                param_grid[p] = [param_grid[p]]
        searchCV = GridSearchCV(method, param_grid, cv=folds, n_jobs=-1, scoring=metric, return_train_score=True)
    
    # Fit the model on the training data
    searchCV.fit(X_train, y_train)
    
    # Test model on test dataset
    test_score =  searchCV.score(X_test, y_test)
    
    return searchCV.best_estimator_, searchCV.best_params_, searchCV.best_score_, searchCV.cv_results_, test_score

In [75]:
def ML_Pipeline(data, json_config):    
    config = json.loads(json_config)
    
    models = {**config["Runs"], **config["Models"]}
    train_config = config["Training"]
    names = list(models.keys())    

    target = train_config["target"]
    
    df = clean_data(data)
    X = df.drop(target, axis=1)
    y = df[target]
    
    data_split = train_test_split(X, y, test_size = train_config['train_test_split'], shuffle=True, random_state=train_config['seed'])
    
    stats = []
    hyperparams = []
    trained_models = [] 

    for model in enumerate(models):
        trained_model, best_params, val_score, cv_results, test_score= train_model(models[model[1]], data_split, train_config)
        
        s = {"validation_score": val_score, "cv_summary": pd.DataFrame(cv_results), "test_score": test_score}
        stats.append(s)
        trained_models.append(trained_model)
        hyperparams.append(best_params)
    return stats, trained_models, hyperparams, names

def get_best_model(stats, trained_models):
    index = np.argmin(stats[1,:])
    best_value = np.min(stats[1,:])
    model= trained_models[index]
    return model,best_value,index

In [81]:
stats, trained_models, hyperparams, names = ML_Pipeline(df, '{"Runs": {"l": {"model_type": "Decision Tree", "params": {"max_depth": [1, 20], "min_samples_split": [2, 15], "min_samples_leaf": [1, 10], "max_features": ["sqrt", "log2"], "max_leaf_nodes": [2, 100], "min_weight_fraction_leaf": [0.0, 0.2], "criterion": ["gini", "entropy", "log_loss"], "splitter": ["best", "random"]}, "n_iter": 5}, "n": {"model_type": "Random Forest Classificator", "params": {"n_estimators": [100, 500], "max_depth": [1, 20], "min_samples_split": [2, 10], "min_samples_leaf": [1, 4], "max_features": ["sqrt", "log2"], "bootstrap": [true, false], "criterion": ["gini", "entropy", "log_loss"]}, "n_iter": 5}, "u": {"model_type": "Logistic Regression", "params": {"penalty": ["l1", "l2", "elasticnet", null], "C": [0.01, 100.0], "solver": ["newton-cg", "liblinear", "sag", "saga"], "max_iter": [100, 1000]}, "n_iter": 5}}, "Models": {"m": {"model_type": "Logistic Regression", "params": {"penalty": null, "C": 0.01, "solver": "newton-cg", "max_iter": 100}}}, "Training": {"is_regression": false, "CV": false, "metric": null, "target": "stroke", "seed": 42, "train_test_split": 0.2}}')
print(names)
print(len(stats))
for i,model_stats in enumerate(stats):
    print("Model " + names[i] + ":")
    print(model_stats["cv_summary"])
            #st.dataframe(model_stats["cv_summary"])
            
    print()
            

c:\Users\Jonathan\miniconda3\envs\streamlit_env\lib\site-packages\sklearn\model_selection\_validation.py:540: FitFailedWarning: 
10 fits failed out of a total of 25.
The score on these train-test partitions for these parameters will be set to nan.
If these failures are not expected, you can try to debug them by setting error_score='raise'.

Below are more details about the failures:
--------------------------------------------------------------------------------
5 fits failed with the following error:
Traceback (most recent call last):
  File "c:\Users\Jonathan\miniconda3\envs\streamlit_env\lib\site-packages\sklearn\model_selection\_validation.py", line 888, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
  File "c:\Users\Jonathan\miniconda3\envs\streamlit_env\lib\site-packages\sklearn\base.py", line 1473, in wrapper
    return fit_method(estimator, *args, **kwargs)
  File "c:\Users\Jonathan\miniconda3\envs\streamlit_env\lib\site-packages\sklearn\linear_model\_logis

['l', 'n', 'u', 'm']
4
Model l:
   mean_fit_time  std_fit_time  mean_score_time  std_score_time  \
0       0.015557      0.004254         0.007362        0.002560   
1       0.014515      0.002406         0.008294        0.002638   
2       0.010871      0.001725         0.007140        0.004096   
3       0.021137      0.017394         0.007457        0.002019   
4       0.013639      0.004079         0.005478        0.000959   

  param_splitter  param_min_weight_fraction_leaf  param_min_samples_split  \
0           best                             0.0                       15   
1           best                             0.2                        2   
2         random                             0.2                        2   
3           best                             0.0                       15   
4           best                             0.0                        2   

   param_min_samples_leaf  param_max_leaf_nodes param_max_features  ...  \
0                      10  

c:\Users\Jonathan\miniconda3\envs\streamlit_env\lib\site-packages\sklearn\linear_model\_logistic.py:1208: UserWarning: Setting penalty=None will ignore the C and l1_ratio parameters
  warnings.warn(


In [387]:
# Evaluate the model on the test set
save_model(trained_models[0], "test_model_1")

Model saved at: saved_models\test_model_1.pkl


In [388]:
t = joblib.load("saved_models/test_model_1.pkl")
print(t)

RandomForestClassifier(criterion='log_loss', max_depth=1, n_estimators=200)
